In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

In [3]:
NUM_CLASSES = 2
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

In [4]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

for layer in base_model.layers[:50]:
    layer.trainable = False
for layer in base_model.layers[50:]:
    layer.trainable = True

def lr_schedule(epoch):
    lr = 0.0001
    if epoch > 5:
        lr *= 0.1
    if epoch > 10:
        lr *= 0.1
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Training Data
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dogs_cat/training_set/training_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Validation Data
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/dogs_cat/test_set/test_set',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 8005 images belonging to 2 classes.
Found 2033 images belonging to 2 classes.


In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/20
250/250 [==============================] - 183s 595ms/step - loss: 4.3900 - accuracy: 0.6819 - val_loss: 3.6529 - val_accuracy: 0.5957 - lr: 1.0000e-04
Epoch 2/20
250/250 [==============================] - 148s 593ms/step - loss: 2.6565 - accuracy: 0.7657 - val_loss: 2.1029 - val_accuracy: 0.8348 - lr: 1.0000e-04
Epoch 3/20
250/250 [==============================] - 148s 592ms/step - loss: 1.7992 - accuracy: 0.8161 - val_loss: 1.4216 - val_accuracy: 0.8735 - lr: 1.0000e-04
Epoch 4/20
250/250 [==============================] - 147s 588ms/step - loss: 1.3496 - accuracy: 0.8298 - val_loss: 1.4707 - val_accuracy: 0.7197 - lr: 1.0000e-04
Epoch 5/20
250/250 [==============================] - 149s 595ms/step - loss: 1.0037 - accuracy: 0.8594 - val_loss: 0.7991 - val_accuracy: 0.9008 - lr: 1.0000e-04
Epoch 6/20
250/250 [==============================] - 148s 589ms/step - loss: 0.7778 - accuracy: 0.8765 - val_loss: 0.6850 - val_accuracy: 0.8805 - lr: 1.0000e-04
Epoch 7/20
250/250 [==

In [7]:
model.save('cat_dog_resnet_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

64/64 [==============================] - 9s 138ms/step - loss: 0.4056 - accuracy: 0.9498
Validation Loss: 0.40559419989585876, Validation Accuracy: 0.9498278498649597
